# Onchain Attest Labels Using Python

### Single Attest

In [ ]:
from web3 import Web3
from eth_abi.abi import encode
import json

# Connect to an Ethereum node
rpc = 'https://sepolia.base.org' # for Base use: https://mainnet.base.org
web3 = Web3(Web3.HTTPProvider(rpc))

# Check if connected to the node
if not web3.is_connected():
    raise Exception("Failed to connect to the Ethereum node")

# Load the EAS contract
from eas_contracts import eas_address, eas_abi
eas = web3.eth.contract(address=eas_address, abi=eas_abi)

# Import your wallet
address = '...' #### fill in here!
private_key = '...' #### fill in here!

# Label Pool Schema v1.0.0
schema = '0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68'

# Input the label in OLI format:
#   chain_id should follow the CAIP-2 standard
#   contract_address should be the address of the contract you want to label
#   tags_json should be a JSON string
chain_id = 'eip155:8453'
contract_address = '0x498581ff718922c3f8e6a244956af099b2652b2b'
ref_uid = '0x0000000000000000000000000000000000000000000000000000000000000000'
tags_json = json.dumps(
                {'contract_name': 'Pool Manager v4',
                'is_eoa': False, 
                'deployment_tx': '0x25f482fbd94cdea11b018732e455b8e9a940b933cabde3c0c5dd63ea65e85349',
                'deployer_address': '0x2179a60856E37dfeAacA0ab043B931fE224b27B6',
                'owner_project': 'uniswap',
                'version': 4,
                'deployment_date': '2025-01-21 20:28:43',
                'source_code_verified': 'https://repo.sourcify.dev/contracts/partial_match/8453/0x498581fF718922c3f8e6A244956aF099B2652b2b/',
                'is_proxy': False}
            )

# ABI encode
encoded_data = encode(['string', 'string'], [chain_id, tags_json])
data = f"0x{encoded_data.hex()}"

# Create the transaction to call the attest function
transaction = eas.functions.attest({
    'schema': web3.to_bytes(hexstr=schema),
    'data': {
        'recipient': web3.to_checksum_address(contract_address),
        'expirationTime': 0,
        'revocable': True,
        'refUID': web3.to_bytes(hexstr=ref_uid),
        'data': web3.to_bytes(hexstr=data),
        'value': 0
    }
}).build_transaction({
    'chainId': 84532, # for Base use: 8453 
    'gas': 1000000,
    'gasPrice': web3.eth.gas_price,
    'nonce': web3.eth.get_transaction_count(address),
})

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, private_key=private_key)

# Send the transaction
txn_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Get the transaction receipt
txn_receipt = web3.eth.wait_for_transaction_receipt(txn_hash)

print(f'Transaction successful with hash: 0x{txn_hash.hex()}')

Transaction successful with hash: 0x6827819f71e9a5758ee20d727db21ad297d29022c992db6254518152f2a11132


### Extract UID from txn_receipt

In [11]:
print('UID: 0x' + txn_receipt.logs[0].data.hex())

UID: 0xab1db82b25611a28b67ec56f918294e3cc6d083cb7e6bae31d8e86990704db76


### Calculate UID

In [12]:
def calculate_attestation_uid(schema, recipient, attester, timestamp, data, expiration_time=0, revocable=True, ref_uid=ref_uid, bump=0):
    # Convert all values to bytes
    schema_bytes = Web3.to_bytes(hexstr=schema)
    recipient_bytes = Web3.to_bytes(hexstr=recipient)
    attester_bytes = Web3.to_bytes(hexstr=attester)
    ref_uid_bytes = Web3.to_bytes(hexstr=ref_uid)
    data_bytes = Web3.to_bytes(hexstr=data)
    revocable_bytes = bytes([1]) if revocable else bytes([0])
    timestamp_bytes = timestamp.to_bytes(8, byteorder='big')
    expiration_bytes = expiration_time.to_bytes(8, byteorder='big')
    bump_bytes = bump.to_bytes(4, byteorder='big') 
    # Pack all the values together in the same order as the Solidity contract
    packed_data = (schema_bytes + recipient_bytes + attester_bytes + timestamp_bytes + expiration_bytes + revocable_bytes + ref_uid_bytes + data_bytes + bump_bytes)
    # Calculate keccak256 hash
    uid = Web3.keccak(packed_data)
    return uid

# get the timestamp from the transaction receipt
timestamp = web3.eth.get_block(txn_receipt['blockNumber'])['timestamp']

# Calculate UID (legacy)
uid = calculate_attestation_uid(schema, contract_address, address, timestamp, data)
uid_hex = '0x' + uid.hex()

print(f"Calculated UID: {uid_hex}")

Calculated UID: 0xab1db82b25611a28b67ec56f918294e3cc6d083cb7e6bae31d8e86990704db76


In [13]:
# getAttestation
eas.functions.getAttestation(web3.to_bytes(hexstr=uid_hex)).call()

(b'\xab\x1d\xb8+%a\x1a(\xb6~\xc5o\x91\x82\x94\xe3\xccm\x08<\xb7\xe6\xba\xe3\x1d\x8e\x86\x99\x07\x04\xdbv',
 b'\xb7c\xe6-\x94\x0b\xedoR}\xd8$\x18\xe1F\xa9\x04\xe6*){\x8f\xa7e\xc9\xb3\xe1\xf0\xbco\xddh',
 1742566148,
 0,
 0,
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 '0x498581fF718922c3f8e6A244956aF099B2652b2b',
 '0x732D31D49467c08F41fD0727537995ea45dD4Ba7',
 True,
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0beip155:8453\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [14]:
# isAttestationValid
eas.functions.isAttestationValid(web3.to_bytes(hexstr=uid_hex)).call()

True

### Single Revoke

In [15]:
# Create the transaction to revoke an attestation
transaction = eas.functions.revoke({
    'schema': web3.to_bytes(hexstr=schema),
    'data': {
        'uid': web3.to_bytes(hexstr=uid_hex),
        'value': 0
    }
}).build_transaction({
    'chainId': 84532, # for Base use: 8453 
    'gas': 1000000,
    'gasPrice': web3.eth.gas_price,
    'nonce': web3.eth.get_transaction_count(address),
})

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, private_key=private_key)

# Send the transaction
txn_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Get the transaction receipt
txn_receipt = web3.eth.wait_for_transaction_receipt(txn_hash)

print(f'Transaction successful with hash: 0x{txn_hash.hex()}')

Transaction successful with hash: 0xa54d82c0c41e6f7cd93c57092c4fab7a95a4ddd0d013b714455daf0307b52c32


### Multi Attest

In [ ]:
from web3 import Web3
from eth_abi.abi import encode
import json

# Connect to an Ethereum node
rpc = 'https://sepolia.base.org' # for Base use: https://mainnet.base.org
web3 = Web3(Web3.HTTPProvider(rpc))

# Check if connected to the node
if not web3.is_connected():
    raise Exception("Failed to connect to the Ethereum node")

# Load the EAS contract
from eas_contracts import eas_address, eas_abi
eas = web3.eth.contract(address=eas_address, abi=eas_abi)

# Import your wallet
address = '...' #### fill in here!
private_key = '...' #### fill in here!

# Label Pool Schema v1.0.0
schema = '0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68'
ref_uid = '0x0000000000000000000000000000000000000000000000000000000000000000'

# Input the labels in OLI format
# First label
chain_id_1 = 'eip155:8453'
contract_address_1 = '0x498581ff718922c3f8e6a244956af099b2652b2b'
tags_json_1 = json.dumps(
    {'contract_name': 'Pool Manager v4',
    'is_eoa': False, 
    'deployment_tx': '0x25f482fbd94cdea11b018732e455b8e9a940b933cabde3c0c5dd63ea65e85349',
    'deployer_address': '0x2179a60856E37dfeAacA0ab043B931fE224b27B6',
    'owner_project': 'uniswap',
    'version': 4,
    'deployment_date': '2025-01-21 20:28:43',
    'source_code_verified': 'https://repo.sourcify.dev/contracts/partial_match/8453/0x498581fF718922c3f8e6A244956aF099B2652b2b/',
    'is_proxy': False}
)
# Second label (add more as needed)
chain_id_2 = 'eip155:8453'
contract_address_2 = '0x25d093633990dc94bedeed76c8f3cdaa75f3e7d5'
tags_json_2 = json.dumps(
    {'contract_name': 'PositionDescriptor',
    'is_eoa': False,
    'deployment_tx': '0x2ee0c3ba9f65f2103b8505f45859b7aed6dfecc2455f4559f18172625d72e63f',
    'deployer_address': '0x2179a60856E37dfeAacA0ab043B931fE224b27B6',
    'owner_project': 'uniswap',
    'version': 4,
    'deployment_date': '2025-01-21 20:28:51',
    'is_proxy': True}
)

# ABI encode the data for each attestation
encoded_data_1 = encode(['string', 'string'], [chain_id_1, tags_json_1])
data_1 = f"0x{encoded_data_1.hex()}"

encoded_data_2 = encode(['string', 'string'], [chain_id_2, tags_json_2])
data_2 = f"0x{encoded_data_2.hex()}"

# Create the multi-attestation request
multi_requests = [{
    'schema': web3.to_bytes(hexstr=schema),
    'data': [
        {
            'recipient': web3.to_checksum_address(contract_address_1),
            'expirationTime': 0,
            'revocable': True,
            'refUID': web3.to_bytes(hexstr=ref_uid),
            'data': web3.to_bytes(hexstr=data_1),
            'value': 0
        },
        {
            'recipient': web3.to_checksum_address(contract_address_2),
            'expirationTime': 0,
            'revocable': True,
            'refUID': web3.to_bytes(hexstr=ref_uid),
            'data': web3.to_bytes(hexstr=data_2),
            'value': 0
        }
    ]
}]

# Create the transaction to call the multiAttest function
transaction = eas.functions.multiAttest(
    multi_requests
).build_transaction({
    'chainId': 84532, # for Base use: 8453 
    'gas': 1500000,  # Increased gas limit for multiple attestations
    'gasPrice': web3.eth.gas_price,
    'nonce': web3.eth.get_transaction_count(address),
})

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, private_key=private_key)

# Send the transaction
txn_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Get the transaction receipt
txn_receipt = web3.eth.wait_for_transaction_receipt(txn_hash)

# Print transaction hash
print(f'Transaction successful with hash: 0x{txn_hash.hex()}')

# Print the UIDs of the attestations
uids = [log.data.hex() for log in txn_receipt.logs]
for i, uid in enumerate(uids):
    print(f'UID {i+1}: 0x{uid}')

Transaction successful with hash: 0xaca924058f2f87d15ee952cbaea90ac1fc6b31d147de1c27f1d1d598ffcd5ced
UID 1: 0x96c9e60d1b4fca3fd1049405619c411de45cf29aa9e0bfa5727bf78661c65691
UID 2: 0xcd54cac21736ac623b914a24a987d11880f0c0faf45291d75d3194aa43db877e


### Multi Revoke

In [17]:
# Dynamically create the revocation data array based on the uids list
revocation_data = []
for uid in uids:
    revocation_data.append({
        'uid': web3.to_bytes(hexstr=uid),
        'value': 0
    })

# Create the multi-revocation request with all UIDs
multi_requests = [{
    'schema': web3.to_bytes(hexstr=schema),
    'data': revocation_data
}]

# Create the transaction to call the multiRevoke function
transaction = eas.functions.multiRevoke(
    multi_requests
).build_transaction({
    'chainId': 84532, # for Base use: 8453 
    'gas': 1000000,  # Increased gas limit for large revocations
    'gasPrice': web3.eth.gas_price,
    'nonce': web3.eth.get_transaction_count(address),
})

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, private_key=private_key)

# Send the transaction
txn_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Get the transaction receipt
txn_receipt = web3.eth.wait_for_transaction_receipt(txn_hash)

# Print transaction hash
print(f'Transaction successful with hash: 0x{txn_hash.hex()}')
print(f'Revoked {len(uids)} attestations')

Transaction successful with hash: 0x2bbfb87c9754cc4aec8f9b98d891fab442c10e56fc26a5b2945988636f4ec1b2
Revoked 2 attestations
